# Section 2: Bug in `discrete_fourier_transform`

The `discrete_fourier_transform` function below is affected by a bug. Subsequently, we implement a unit test, Differential Testing, Metamorphic Testing, and Retromorphic Testing to identify the bug.

In [4]:
import numpy as np

def discrete_fourier_transform(sequence, opt = 1):
    N = len (sequence)
    sequence_new = [0 for _ in range ( N )]
    for k in range (N):
        for n in range (N):
            sequence_new [k] += sequence [n] * np.exp (opt * -1j * np.pi / N * k * n) # Bug: should be -2j
    if(opt == -1):
        for i in range(len(sequence_new)):
            sequence_new[i] = sequence_new[i] / N
    return sequence_new

## Listing 1: Unit Testing

A manually-written unit test case could identify the bug. However, since writing test cases manually incurs significant time and effort, we subsequently consider automated approaches.

In [5]:
time_sequence = [2 , 0 , 1 , 0]
frequency_sequence = discrete_fourier_transform(time_sequence)
frequency_sequence = [int(_.real) for _ in frequency_sequence]
assert frequency_sequence == [2 , 0 , 2 , 0]

AssertionError: 

## Listing 2: Differential Testing

To realize a Differential Testing approach, we require multiple components that produce the same output for the given input. Below, we assume the existence of alternative transformation function, whose output we can compare.

In [6]:
import random 

def get_random_sequence(len = 4):
    return [2, 0, 2, 1]
    '''
    sequence = [random.random() for _ in range(len)]
    return sequence
    '''
def fast_fourier_transform(sequence, opt = 1):
    N = len(sequence)
    if N <= 1:
        return sequence
    even = fast_fourier_transform(sequence[0::2])
    odd = fast_fourier_transform(sequence[1::2])
    T = [np.exp(opt * -2j * np.pi * k / N) * odd[k] for k in range(N // 2)]
    return [even[k] + T[k] for k in range(N // 2)] + [even[k] - T[k] for k in range(N // 2)]



By checking whether the frequency sequence are equal, we can identify the bug in the `discrete_fourier_transform` implementation.

In [7]:
eps = 1e-10
transform_algorithms = [discrete_fourier_transform, fast_fourier_transform]
while True :
    time_sequence = get_random_sequence()
    frequency_sequences = [alg(time_sequence.copy()) for alg in transform_algorithms]
    frequency_sequences = [[_.real for _ in frequency_sequence] for frequency_sequence in frequency_sequences]
    all_same = all(all(abs(frequency_sequence[i] - frequency_sequences[0][i]) < eps for i in range(len(frequency_sequence))) for frequency_sequence in frequency_sequences)
    assert all_same

AssertionError: 

# Listing 3: Metamorphic Testing

The core idea of Metamorphic Testing is to generate a follow-up test case whose output we can infer based on the original test case's output. To test our `discrete_fourier_transform` implementation, we check whether the relative difference is maintained when a constant is added to the first element in time sequence. However, this metamorphic testing approach failed to find this bug.

In [8]:
eps = 1e-10
while True :
    time_sequence = get_random_sequence() 
    frequency_sequence = discrete_fourier_transform(time_sequence)
    c = random.random()
    time_sequence[0] += c
    frequency_sequence_new = discrete_fourier_transform(time_sequence)
    all_same = all(abs(frequency_sequence_new[i].real - frequency_sequence[i].real - c) < eps for i in range(len(frequency_sequence))) 
    assert all_same

KeyboardInterrupt: 

# Listing 4: Retromorphic Testing

The core idea of Retromorphic Testing is to add a forward or backward program, so that we can test a relation that is expected to hold for the original input and the backward program's output. Here, we use the inverse option of `discrete_fourier_transform` and validate that this function's inversed output equals the original input, which allows us to find the bug as well.

In [9]:
eps = 1e-10
while True:
    time_sequence = get_random_sequence()
    frequency_sequence = discrete_fourier_transform(time_sequence)
    
    #Without Modification
    time_sequence_new = discrete_fourier_transform(frequency_sequence, -1)
    all_same = all(abs(time_sequence_new[i].real - time_sequence[i].real) < eps for i in range(len(time_sequence))) 
    assert all_same
    
    #With Modification
    c = random.random()
    for i in range(len(frequency_sequence)):
        frequency_sequence[i] += c
    time_sequence_new = discrete_fourier_transform(frequency_sequence, -1)
    all_same = all(abs(time_sequence_new[i].real - time_sequence[i].real) < eps for i in range(1, len(time_sequence)) and abs(time_sequence_new[0].real - time_sequence[0].real) < eps) 
    assert all_same

AssertionError: 

# Section 4: Examples

## Example 1: Interger Factorization

In this example, we assume `pollards_rho` returns the factorization of a given interger and it is affected by a bug.

In [10]:
import random

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def pollards_rho(n):
    if n == 1:
        return []
    if n % 2 == 0:
        return [2] + pollards_rho(n // 2)
    
    def rho(x, c):
        return (x * x + c) % n
    
    x = random.randint(1, n - 1)
    y = x
    c = random.randint(1, n - 1)
    d = 1
    while d <= 1:
        x = rho(x, c)
        y = rho(rho(y, c), c)
        d = gcd(abs(x - y), x) # Bug: should be gcd(abs(x - y), n)
    if d == n:
        return [n]
    else:
        return pollards_rho(d) + pollards_rho(n // d)

To realize a retromorphic testing technique, we multiply those given factors and check if it is the same as the input integer.

In [11]:
def get_random_integer():
    return 12
    '''
    return random.randint(1,1000)
    '''

In [16]:
while True:
    N = get_random_integer()
    factors = pollards_rho(N)
    result = 1
    for prime in factors:
        result *= prime
    assert(result == N)

AssertionError: 

## Example 2: Infix, Prefix, and Postfix Transformation

In this example, we assume prefix and postfix notation are implement in `string` format. The `postfix_to_prefix` and `prefix_to_postfix` and functions to transform notation between prefix and postfix. The function `postfix_to_prefix` is affected by a bug.

In [13]:
def is_operator(char):
    return char in "+-*/"

def postfix_to_prefix(postfix_expression):
    stack = []
    
    for char in postfix_expression:
        if char.isalnum(): 
            stack.append(char)
        elif is_operator(char):
            operand1 = stack.pop() 
            operand2 = stack.pop() # Bug: should swap operand1 and operand2
            prefix = f"{char}{operand1}{operand2}"
            stack.append(prefix)
    
    return stack[0]

def prefix_to_postfix(prefix_expression):
    stack = []

    for char in reversed(prefix_expression):
        if char.isalnum():  
            stack.append(char)
        elif is_operator(char):
            operand1 = stack.pop()
            operand2 = stack.pop()
            postfix = f"{operand1}{operand2}{char}"
            stack.append(postfix)

    return stack[0]

To realize a retromorphic testing technique, we transform a postfix notation to its prefix notation and convert it back. 

In [14]:
def get_random_postfix():
    return '56a*+'

In [15]:
while True:
    postfix_expression = get_random_postfix()
    assert(postfix_expression == prefix_to_postfix(postfix_to_prefix(postfix_expression)))

AssertionError: 

## Example 5: Machine Translation

We are using Google translate api in this example

M1 (The input English setnece): Mike will leave the bloc if Jack leaves the city.

M2 (The output Chinese setnece): 如果杰克离开这座城市，迈克就会离开欧盟。

M2' (The input Chinese setnece): 如果杰克离开这座城市，迈克就会离开欧盟。

M1' (The output English setnece): Mike will leave the EU if Jack leaves the city.

In [ ]:
def translate_text(text, target_language = 'en'):
    pass
    # use your api to invoke google translate

text_m1 = "Mike will leave the bloc if Jack leaves the city."
text_m2 = translate_text(text_m1, 'zh-cn')
text_m1_ = translate_text(text_m2, 'en')




## Example 6: Image captioning and generation

We are using DALL-E2 as the Image generation system and Microsoft Azure Cognitive Services api for Image caption.

M1 (The input discription): We are looking up at the underside of an airplane.

M2 (The output image):

<img src="image/m2.jpg" alt="Alt Text" width="250" height="250">

M2' (The filtered image): 

<img src="image/m2-filter.jpg" alt="Alt Text" width="250" height="250">

M1' (The output caption): A close-up of a white cylinder. 

We use the following code to invoke the Microsoft Azure Cognitive Services to caption the filtered image, you can reproduce the result by filling your own subscription key

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

class AzureI2C:
    def Image2Caption_url(self, image_url):
        '''
        Return text caption generated from the image_url.
        '''     
        subscription_key = "" # please fill it by your key
        endpoint = "https://compvisionresourcing.cognitiveservices.azure.com/"
        computervision_client = ComputerVisionClient(
            endpoint, CognitiveServicesCredentials(subscription_key))

        description_results = computervision_client.describe_image(image_url)
        if (len(description_results.captions) == 0):
           image_caption = "No description detected."
        else:
            image_caption = description_results.captions[0].text
        return image_caption

    def Image2Caption_path(self, image_path):
        '''
        Return text caption generated from the image_path.
        '''
        subscription_key = "" # please fill it by your key
        endpoint = "https://compvisionresourcing.cognitiveservices.azure.com/"
        computervision_client = ComputerVisionClient(
            endpoint, CognitiveServicesCredentials(subscription_key))

        with open(image_path, "rb") as image:
            description_results = computervision_client.describe_image_in_stream(image)
        if (len(description_results.captions) == 0):
            image_caption = "No description detected."
        else:
            image_caption = description_results.captions[0].text
        return image_caption

def unit_testing():
    Tester = AzureI2C()
    caption = Tester.Image2Caption_path("./image/m2-filter.jpg")
    print(caption)

if __name__ == "__main__":
    unit_testing()